# 雲端載入資料

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#下載yolov5

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 14308, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 14308 (delta 30), reused 39 (delta 17), pack-reused 14242
Receiving objects: 100% (14308/14308), 13.57 MiB | 18.37 MiB/s, done.
Resolving deltas: 100% (9837/9837), done.


#下載Data並放入指定路徑資料夾

In [ ]:
!unzip -qq /content/drive/MyDrive/yolov5/data/Training_Dataset_v5.zip
!unzip -qq /content/drive/MyDrive/yolov5/data/Public_Testing_Dataset_v2.zip
!unzip -qq /content/drive/MyDrive/data.zip

In [ ]:
!unzip -qq /content/drive/MyDrive/yolov5/data/Private_Testing_Dataset_v2.zip

In [ ]:
import os
os.mkdir('/content/train/images')
os.mkdir('/content/train/labels')
os.mkdir('/content/val')
os.mkdir('/content/val/images')
os.mkdir('/content/val/labels')

#資料前處理-處理成yolov5對應之格式

In [ ]:
import numpy as np
def retxt(path, height, width):
  if os.stat(path).st_size == 0:
    return
  data = open(path)
  list_of_lists = []
  for line in data:
    stripped_line = line.strip()
    ob_list = [np.float(s) for s in stripped_line.split(',')]
    ob_list[0] = np.int32(ob_list[0])
    ob_list[1] = (ob_list[1]+(ob_list[3]/2))/width
    ob_list[2] = (ob_list[2]+(ob_list[4]/2))/height
    ob_list[3] = ob_list[3]/width
    ob_list[4] = ob_list[4]/height
    list_of_lists.append(ob_list)
  np.savetxt(path, list_of_lists, fmt='%d %f %f %f %f', delimiter=' ', newline='\n')

In [ ]:
import cv2

for i in range(1, 801, 1):
  if(i<10):
    img = "img000" + str(i) + ".png"
    text = "img000" + str(i) + ".txt"
  elif(i<100):
    img = "img00" + str(i) + ".png"
    text = "img00" + str(i) + ".txt"
  else:
    img = "img0" + str(i) + ".png"
    text = "img0" + str(i) + ".txt"
  height, width, _ = cv2.imread('/content/train/' + str(img)).shape
  retxt('/content/train/' + str(text), height, width)
  os.replace('/content/train/' + str(img), '/content/train/images/' + str(img))
  os.replace('/content/train/' + str(text), '/content/train/labels/' + str(text))
for i in range(801, 1001, 1):
  if(i<1000):
    img = "img0" + str(i) + ".png"
    text = "img0" + str(i) + ".txt"
  else:
    img = "img" + str(i) + ".png"
    text = "img" + str(i) + ".txt"
  height, width, _ = cv2.imread('/content/train/' + str(img)).shape
  retxt('/content/train/' + str(text), height, width)
  os.replace('/content/train/' + str(img), '/content/val/images/' + str(img))
  os.replace('/content/train/' + str(text), '/content/val/labels/' + str(text))

#yolov5x模型訓練

##yolov5x yaml檔之nc要改為class數量

In [ ]:
!python /content/yolov5/train.py --data ./data.yaml --cfg /content/yolov5/models/yolov5x.yaml --weights '/content/drive/MyDrive/yolov5/reanchor-300/weights/last.pt' --batch-size -1 --epochs 50 --rect --imgsz 1080 --image-weights

train: weights=/content/yolov5/runs/train/exp4/weights/last.pt, cfg=/content/yolov5/models/hub/yolov5s6.yaml, data=./data.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=-1, imgsz=1080, rect=True, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=True, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-5-gbfa1f23 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0

In [ ]:
import shutil
#儲存model至雲端硬碟
shutil.copytree('/content/yolov5/runs/train/exp', '/content/drive/MyDrive/yolov5/reanchor-350')

'/content/drive/MyDrive/yolov5/reanchor-350'

#測試model(單張圖)

In [ ]:
import torch
model = torch.hub.load('ultralytics/yolov5', 'custom' , path='/content/drive/MyDrive/yolov5/reanchor-350/weights/best.pt')

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-11-24 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
YOLOv5x summary: 322 layers, 86193601 parameters, 0 gradients, 203.8 GFLOPs
Adding AutoShape... 


In [ ]:
result = model('/content/public/img1022.png')

In [ ]:
from PIL import Image
r_img = result.render()[0]
r_img = Image.fromarray(r_img)
r_img.show()
#r_img = r_img.resize((960, 540))
#r_img = np.array(r_img)
#print(r_img.shape)
#print(r_img)

In [ ]:
result #yolov5x last epoch reanchor 250

YOLOv5 <class 'models.common.Detections'> instance
image 1/1: 1080x1920 23 cars, 2 hovs, 18 motorcycles
Speed: 80.6ms pre-process, 61.8ms inference, 2.3ms NMS per image at shape (1, 3, 384, 640)

#testing data預測

In [ ]:
!python /content/yolov5/detect.py --weights /content/drive/MyDrive/yolov5/reanchor-350/weights/best.pt --img 1080 --save-txt --source /content/Private_Testing_Dataset_v2 --conf 0.5

detect: weights=['/content/drive/MyDrive/yolov5/reanchor-350/weights/best.pt'], source=/content/Private_Testing_Dataset_v2, data=yolov5/data/coco128.yaml, imgsz=[1080, 1080], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
requirements: YOLOv5 requirements "gitpython" "ipython" not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

requirements: 2 packages updated per /content/yolov5/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 v7.0-21-ga1b6e79 Python-3.8.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
YOLOv5x summary: 322 

#預測目標之csv

In [ ]:
import csv
import cv2
import numpy as np
from os import listdir

with open('Private_Testing.csv', 'w', newline='') as csvfile:
  writer = csv.writer(csvfile)
  for text in listdir('/content/yolov5/runs/detect/exp/labels/'):
    height, width, _ = cv2.imread('/content/yolov5/runs/detect/exp/' + text.replace('.txt','') + '.png').shape
    data = open('/content/yolov5/runs/detect/exp/labels/' + text.replace('.txt','') + '.txt')
    for line in data:
      stripped_line = line.strip()
      ob_list = [np.float(s) for s in stripped_line.split(' ')]
      ob_list[1] = (ob_list[1]-(ob_list[3]/2))*width
      ob_list[2] = (ob_list[2]-(ob_list[4]/2))*height
      ob_list[3] = ob_list[3]*width
      ob_list[4] = ob_list[4]*height
      writer.writerow([text.replace('.txt',''), np.int32(ob_list[0]), np.int32(ob_list[1]), np.int32(ob_list[2]), np.int32(ob_list[3]), np.int32(ob_list[4])])

<ipython-input-9-78ddca96cee3>:13: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  ob_list = [np.float(s) for s in stripped_line.split(' ')]
